In [130]:
import matplotlib.pyplot as plt
import os
import numpy as np
import json
from skimage import io
from skimage.filters import roberts, sobel, scharr, prewitt

In [131]:
data_dir = 'Trainfile path'
validation_dir= 'testfilepath'

#test_dir='test/'
validation_cut = 0.2
fov=1.0
#fov = 0.33

In [132]:
import math
def foveate(image, fov = fov):
    if not fov:
        return image
    centerx = image[1].shape[1]/2
    centery = image[1].shape[2]/2
    fov_pos = []
    for x,y in image[0]['XY']:
        if (math.sqrt((x-centerx)**2+(y-centery)**2) < fov * math.sqrt(centerx**2+centery**2)):
            fov_pos.append([x,y])
    image[0]['XY'] = fov_pos
    return image

In [133]:
# Load training data
import matplotlib.pyplot as plt
import os
import numpy as np
import json
from skimage import io

print("Indexing data directory")

images = []

for filename in os.listdir(data_dir):
    if filename.endswith(".tif") and \
    os.path.isfile(os.path.join(data_dir, filename +'.json')):
        
        img = io.imread(os.path.join(data_dir, filename))
        with open(os.path.join(data_dir, filename +'.json'), 'r') as fp:
            d = json.load(fp)
        images.append(foveate([d, img]))
        print(images)
        
       # print("Loaded " + filename)

Indexing data directory


In [134]:
import random
import copy

validationimages = []

if validation_dir:

    print("Indexing validation data directory")

    for filename in os.listdir(validation_dir):
        if filename.endswith(".tif") and \
        os.path.isfile(os.path.join(validation_dir, filename +'.json')):

            validationimg = io.imread(os.path.join(validation_dir, filename))
            with open(os.path.join(validation_dir, filename +'.json'), 'r') as fp:
                d = json.load(fp)
            validationimages.append(foveate([d, validationimg]))
            print("Loaded " + filename)
else:
    print("Doing train/test split")
    ntrain = 0
    ntest = 0
    for i in range(len(images)):
        train = []
        test = []
        shuffled = images[i][0]['XY']
        random.shuffle(shuffled)
        frac = int(math.ceil(len(shuffled)*validation_cut))
        test = shuffled[:frac]
        ntest += len(test)
        train = shuffled[frac:]
        ntrain += len(train)
        valimg = copy.deepcopy(images[i])
        images[i][0]['XY'] = train
        valimg[0]['XY'] = test
        validationimages.append(valimg)
    print("Total of {} training, {} testing spots...".format(ntrain, ntest))
        

Indexing validation data directory
Loaded 11.30.21_SPP_Fov8_1_NDTiffStack (1).tif


In [135]:
print(foveate([d, validationimg]))

[{'XY': [[307.6484266469546, 386.765335253559]], 'Z': {'0': '17.98440497622342', '1': '17.936170211174467', '2': '17.88636491424136', '3': '17.83852554134603', '4': '17.788919237628747', '5': '17.73086675690216', '6': '17.681754355188545', '7': '17.635589843000787', '8': '17.583729759410595', '9': '17.52764779828279', '10': '17.484629821947763', '11': '17.436604045462563', '12': '17.390347222078606', '13': '17.334958356547272', '14': '17.28683811904829', '15': '17.234300057152655', '16': '17.177933714013246', '17': '17.131192150445617', '18': '17.08749570885637', '19': '17.03083910965764', '20': '16.98223555966236', '21': '16.930687684957558', '22': '16.88405032504755', '23': '16.831916009005475', '24': '16.783513851220224', '25': '16.73167678883835', '26': '16.68789127110267', '27': '16.635565940720742', '28': '16.584420044887725', '29': '16.537005778971896', '30': '16.485666184273985', '31': '16.434916922280575', '32': '16.380733504256064', '33': '16.333324267426516', '34': '16.28601

In [136]:
def tile_generator(images,bs, input_size, shift, shuffle = True):
    if shift<1:
        shift = int(input_size*shift)
       # print(images)
        
    n_stacks = len(images)
    c_stack = c_spot = c_depth = 0
    
    while True:
        inputs = []
        targets = []
        while len(inputs) < bs:
            if shuffle:
                stack = random.choice(images)
                X, Y = random.choice(stack[0]['XY'])
                c = random.choice(list(stack[0]['Z'].items()))
                imgnum = int(c[0])
               # print(imgnum)
                Z = float(c[1])
            else:
                stack = images[c_stack]
                X, Y = stack[0]['XY'][c_spot]
                c = list(stack[0]['Z'].items())[c_depth]
                imgnum = int(c[0])
                Z = float(c[1])
                
                n_spots = len(stack[0]['XY'])
                n_depths = len(stack[0]['Z'])
                
                c_depth += 1
                
                # Move to next spot if all depths have been consumed
                if c_depth == n_depths:
                    c_depth = 0
                    c_spot += 1
                    
                    # Move to next stack if all spots have been consumed
                    if c_spot == n_spots:
                        c_spot = 0
                        c_stack += 1
                        
                        # Start a new epoch if all stacks have been consumed
                        if c_stack == n_stacks:
                            c_stack = 0
            
            xcrop, ycrop = tuple(int(p + random.uniform(-1*shift, shift)) for p in (X, Y))
            img = stack[1][imgnum][ycrop-input_size//2:ycrop+(input_size-input_size//2), \
                                  xcrop-input_size//2:xcrop+(input_size-input_size//2)]

            x_train = (X - (xcrop-input_size//2))/input_size
            #print(x_train)
            y_train = (Y - (ycrop-input_size//2))/input_size
            z_vals = [float(v) for v in list(stack[0]['Z'].values())]
            z_train = (Z - min(z_vals)) / ((max(z_vals)-min(z_vals))/2) - 1
            try:
                # normalize image
                img = np.divide(img,np.max(img))
                img.reshape(input_size, input_size, 1)
            except:
                import pdb;pdb.set_trace()
            inputs.append(img.reshape(input_size, input_size, 1))
            targets.append(np.array([x_train, y_train, z_train]))
            #print(x_train)
        yield(np.array(inputs),np.array(targets))

In [137]:
# Instantiate generators

input_size = 64
datagen = tile_generator(images, 32, input_size, .2)
validationdatagen = tile_generator(validationimages, 32, input_size, .2)
#print(datagen)

In [11]:
# compile model, based on https://www.pyimagesearch.com/2019/01/28/keras-regression-and-cnns/
from keras.models import Sequential
from keras.layers import *

model = Sequential()

model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=(input_size,input_size,1)))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(BatchNormalization(axis=-1))
model.add(Dropout(.5))

model.add(Dense(3, activation ='tanh'))

model.compile(loss='mse',
              optimizer='adadelta', metrics=['accuracy'])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 16)        160       
                                                                 
 batch_normalization (BatchN  (None, 62, 62, 16)       64        
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        4640      
                                                                 
 batch_normalization_1 (Batc  (None, 29, 29, 32)       128       
 hNormalization)                                                 
                                                        

In [13]:
# train and visualize loss

history = model.fit_generator(datagen,validation_data=validationdatagen, validation_steps=100,steps_per_epoch=100, epochs=50)

Epoch 1/50


C:\Users\Bharath\AppData\Local\Temp\ipykernel_20200\2818746364.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(datagen,validation_data=validationdatagen, validation_steps=100,steps_per_epoch=100, epochs=50)


100/100 [==============================] - 7s 63ms/step - loss: 0.8059 - accuracy: 0.3475 - val_loss: 0.2391 - val_accuracy: 0.3038
Epoch 2/50
100/100 [==============================] - 7s 68ms/step - loss: 0.8079 - accuracy: 0.3506 - val_loss: 0.2877 - val_accuracy: 0.2700
Epoch 3/50
100/100 [==============================] - 6s 62ms/step - loss: 0.7850 - accuracy: 0.3519 - val_loss: 0.2797 - val_accuracy: 0.2691
Epoch 4/50
100/100 [==============================] - 7s 66ms/step - loss: 0.7715 - accuracy: 0.3678 - val_loss: 0.2777 - val_accuracy: 0.2587
Epoch 5/50
100/100 [==============================] - 7s 66ms/step - loss: 0.7673 - accuracy: 0.3841 - val_loss: 0.3261 - val_accuracy: 0.2766
Epoch 6/50
100/100 [==============================] - 7s 68ms/step - loss: 0.7685 - accuracy: 0.3791 - val_loss: 0.4240 - val_accuracy: 0.3041
Epoch 7/50
100/100 [==============================] - 7s 70ms/step - loss: 0.7564 - accuracy: 0.3650 - val_loss: 0.5253 - val_accuracy: 0.3225
Epoch 8/50

In [ ]:
model.evaluate_generator(datagen, 32)


In [138]:
print(len(images))

84


In [139]:
# show predictions and write results
import csv
shift=.2
#print(validationimages)
stack = random.choice(validationimages)
#print(len(stack))
X, Y = random.choice(stack[0]['XY'])
print(X)
n_spots = len(stack[0]['XY'])
xcrop, ycrop = tuple(int(p + random.uniform(-1*shift, shift)) for p in (X, Y))
#print(n_spots)
n_depths = len(stack[0]['Z'])
#print(n_depths)
z_vals = [float(v) for v in list(stack[0]['Z'].values())]           

testdatagen = tile_generator(validationimages, n_spots*n_depths, input_size, 0, False)
#testdatagen = tile_generator(validationimages, 50, input_size, 0, True)


batch = next(testdatagen)
#print(len(batch[0][1]))
print('Predicting ' + str(len(batch[0])) + " images...")

cumulative_error = 0

plot = False

%matplotlib inline

with open('res.csv', 'w', newline='') as csvfile:
    
    # Instantiate writer and write row names
    reswriter = csv.writer(csvfile)
    reswriter.writerow(['x_train', 'y_train', 'z_train', \
                        'x_pred', 'y_pred','z_pred'])
    #print(batch[0])
    for samplenum in range(len(batch[0])):
        #print(len(batch[0]))
        #print(batch)
        predictions = model.predict_on_batch(batch[0])
        #print(predictions) 
        #print(samplenum)
        img = batch[0][samplenum][:,:,0]
        #print(img)
        x_train, y_train, z = batch[1][samplenum]
        #inverse Normalization
        X_train = x_train * input_size + (xcrop - input_size // 2)
        Y_train = y_train * input_size + (ycrop - input_size // 2)
        #print(samplenum)    
     
        x_pred, y_pred, z_pred = predictions[samplenum]
        X = x_pred * input_size + (xcrop - input_size // 2)
        Y = y_pred * input_size + (ycrop - input_size // 2)
        Z_pred = (z_pred + 1) / 2 * ((max(z_vals) - min(z_vals)) + min(z_vals))
        
        
        #print(x_pred)
        z_vals = [float(v) for v in list(images[0][0]['Z'].values())]
        z,z_pred = [(d + 1) * ((max(z_vals)-min(z_vals))/2) + min(z_vals) for d in [z,z_pred]]
        
        cumulative_error += abs(z-z_pred)
        
        # Write row to CSV
        reswriter.writerow([X_train, Y_train, z, \
                           X, Y, z_pred])
        
        

        #print("target z:" + str(z) + '\npredicted z:' + str(z_pred))

        if plot:
            plt.figure()
            plt.imshow(img)
            plt.plot(input_size*x_train, input_size*y_train, 'bo')
            plt.plot(input_size*x_pred, input_size*y_pred, 'rX')
            plt.axis('off')
            plt.show()

avg_error = cumulative_error/len(batch[0])
print("Average error in z: {}".format(avg_error))

307.6484266469546
Predicting 122 images...
Average error in z: 1.3400540868918005


In [ ]:
stack = random.choice(validationimages)

n_spots = len(stack[0]['XY'])
n_depths = len(stack[0]['Z'])
#print(n_spots)
testdatagen = tile_generator(images, n_spots*n_depths, input_size, 0, False)

batch = next(testdatagen)
#print(batch)
print(stack)

In [ ]:
validation_loss, val_acc = model.evaluate_generator(validationdatagen, steps=32)
print('Train: %.3f, Test: %.3f' % (validation_loss, val_acc))

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()